In [1]:
using CPTVCA
using ExactDiagonalization
using QuantumLattices
using Distributed
using Plots
using LinearAlgebra

In [3]:
tri = Lattice([0, 0]; vectors=[[1, 0],[-1/2, √3/2]])
sq = Lattice([0, 0]; vectors=[[1, 0],[0, 1]])

Lattice(lattice)
  with 1 point:
    [0.0, 0.0]
  with 2 translation vectors:
    [1.0, 0.0]
    [0.0, 1.0]


In [4]:
function ek(nv::AbstractVector, t::Real, k::AbstractVector)
    ek = 0.0
    for i in eachindex(nv)
        ek += (exp(im*dot(k, nv[i])) + exp(-im*dot(k, nv[i])))
    end
    return t*ek.re
end
function tbae(nv::AbstractVector, t::Real, rz::AbstractVector, ff::Real)
    e = Vector{Float64}(undef, length(rz))
    for i in eachindex(rz)
        e[i] = ek(nv, t, rz[i])
    end
    ne = sort(e)
    return sum(ne[1:Int(length(rz)*ff)])/length(ne[1:Int(length(rz)*ff)])
end
ttrbz = ReciprocalZone(reciprocals(tri.vectors); length=100)
tsqbz = ReciprocalZone(reciprocals(sq.vectors); length=100)
ttba = tbae([[1,0],[1/2,√3/2],[-1/2,√3/2]], -1, ttrbz, 1//2)
stba = tbae([[1,0],[0,1]], -1, tsqbz, 1//2)

-1.6208723769030133

In [7]:
ttba

-1.9765510227181091

In [8]:
stba

-1.6208723769030133

In [12]:
trc = Lattice(tri,(2,2),('p','p'))
#trc = Lattice([0, 0],[1, 0], [1/2, √3/2];vectors=[[3/2,√3/2],[0,√3]])
trbz = ReciprocalZone(reciprocals(trc.vectors); length=100)
hilbert = Hilbert(site=>Fock{:f}(1, 2) for site=1:length(trc))
cons= Conservation(4, 1//2; np=4, sz=0)
t = Hopping(:t, Complex(-1.0), 1)
origiterms = (t,)
referterms = (t,)
neighbors = Neighbors(0=>0.0, 1=>1.0)
trvca = VCA(tri, trc, hilbert, origiterms, referterms, cons; neighbors=neighbors, m=200)
trgp = GrandPotential(trvca, trbz, 0)
trgp[1]

-2.06500566677399

In [6]:
trc = Lattice([0.0, 0.0], [1, 0], [-1/2, √3/2], [1/2, √3/2], [3/2, √3/2], [-1, √3], [0, √3], [1, √3], [2, √3], [-1/2, 3√3/2], [1/2, 3√3/2], [3/2, 3√3/2]; vectors = [[3, √3], [-3, √3]])
trbz = ReciprocalZone(reciprocals(trc.vectors); length=100)
hilbert = Hilbert(site=>Fock{:f}(1, 2) for site=1:length(trc))
cons= Conservation(12, 1//2; np=12, sz=0)
t = Hopping(:t, Complex(-1.0), 1)
origiterms = (t,)
referterms = (t,)
neighbors = Neighbors(0=>0.0, 1=>1.0)
trvca = VCA(tri, trc, hilbert, origiterms, referterms, cons; neighbors=neighbors, m=200)
trgp = GrandPotential(trvca, trbz, 0)

(-2.0650233956126196, -5.057641179973375, 0.0)

In [9]:
trgp[1]

-2.0650233956126196

In [13]:
sqc = Lattice(sq,(2,2),('p','p'))
sqbz = ReciprocalZone(reciprocals(sqc.vectors); length=100)
hilbert = Hilbert(site=>Fock{:f}(1, 2) for site=1:length(sqc))
cons= Conservation(4, 1//2; np=4, sz=0)
t = Hopping(:t, Complex(-1.0), 1)
origiterms = (t,)
referterms = (t,)
neighbors = Neighbors(0=>0.0, 1=>1.0)
sqvca = VCA(sq, sqc, hilbert, origiterms, referterms, cons; neighbors=neighbors, m=200)
sqgp = GrandPotential(sqvca, sqbz, 0) 
sqgp[1]

-1.621072277212208

In [14]:
sqc = Lattice(sq,(2,4),('p','p'))
sqbz = ReciprocalZone(reciprocals(sqc.vectors); length=100)
hilbert = Hilbert(site=>Fock{:f}(1, 2) for site=1:length(sqc))
cons= Conservation(8, 1//2; np=8, sz=0)
t = Hopping(:t, Complex(-1.0), 1)
origiterms = (t,)
referterms = (t,)
neighbors = Neighbors(0=>0.0, 1=>1.0)
sqvca = VCA(sq, sqc, hilbert, origiterms, referterms, cons; neighbors=neighbors, m=200)
sqgp = GrandPotential(sqvca, sqbz, 0) 
sqgp[1]

-1.621122275760784

In [16]:
#square
r1 = -1.6208723769030133 - (-1.621072277212208)#L=4
r2 = -1.6208723769030133 - (-1.621122275760784)#L=8
(r1, r2)

(0.00019990030919458945, 0.00024989885777060294)

In [18]:
#triangle
r1 = -1.9765510227181091 - (-2.06500566677399)#L=4
r2 = -1.9765510227181091 - (-2.0650233956126196)#L=12
(r1, r2)

(0.08845464405588088, 0.08847237289451049)